# Exam Score Prediction using Linear Regression

## Project Overview
This project predicts exam scores using various student features including age, study hours, class attendance, sleep quality, and other factors. We'll implement linear regression from scratch using NumPy.

## Dataset
- **File**: Exam_Score_Prediction.csv
- **Samples**: 234 student records
- **Target**: exam_score
- **Features**: 12 input features including demographic, behavioral, and environmental factors

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Set random seed for reproducibility
np.random.seed(50)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

## Step 1: Load and Explore Data

In [ ]:
# Load the dataset
df = pd.read_csv('../Exam_Score_Prediction.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nFirst Few Rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nStatistical Summary:")
print(df.describe())

In [ ]:
# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Check data types
print("\nData Types:")
print(df.dtypes)

# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nCategorical Columns: {categorical_cols}")
print(f"Numerical Columns: {numerical_cols}")

## Step 2: Data Preprocessing and Feature Encoding

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Separate features and target
X = df_processed.drop('exam_score', axis=1)
y = df_processed['exam_score'].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Identify columns to encode
print("\nUnique values in categorical columns:")
for col in categorical_cols:
    if col in X.columns:
        print(f"{col}: {X[col].nunique()} unique values - {X[col].unique()[:5]}")

In [ ]:
# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        label_encoders[col] = le
        print(f"Encoded {col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")

print("\nFeatures after encoding:")
print(X.head())

In [ ]:
# Convert to numpy arrays
X = X.values.astype(np.float32)
y = y.astype(np.float32)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("\nFeatures normalized (mean ≈ 0, std ≈ 1)")
print(f"Training features mean: {X_train.mean(axis=0)[:5].round(3)}")
print(f"Training features std: {X_train.std(axis=0)[:5].round(3)}")

## Step 3: Implement Linear Regression from Scratch using NumPy

In [ ]:
class LinearRegressionNumPy:
    """
    Linear Regression implementation using NumPy.
    Uses the Normal Equation method: β = (X^T X)^-1 X^T y
    """
    
    def __init__(self):
        self.coefficients = None
        self.intercept = None
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """
        Fit the linear regression model using the Normal Equation.
        
        Parameters:
        X: Training features (n_samples, n_features)
        y: Training target values (n_samples,)
        """
        # Add bias term (column of ones) to X
        X_with_bias = np.column_stack([np.ones(X.shape[0]), X])
        
        # Normal Equation: β = (X^T X)^-1 X^T y
        # Calculate X^T X
        XTX = X_with_bias.T @ X_with_bias
        
        # Calculate X^T y
        XTy = X_with_bias.T @ y
        
        # Solve for coefficients: β = (X^T X)^-1 X^T y
        coefficients = np.linalg.solve(XTX, XTy)
        
        # Store intercept and coefficients
        self.intercept = coefficients[0]
        self.coefficients = coefficients[1:]
        
        self.X_train = X
        self.y_train = y
        
        print(f"Model trained successfully!")
        print(f"Intercept: {self.intercept:.4f}")
        print(f"Number of features: {len(self.coefficients)}")
        
        return self
    
    def predict(self, X):
        """
        Make predictions on new data.
        
        Parameters:
        X: Features (n_samples, n_features)
        
        Returns:
        Predicted values
        """
        if self.coefficients is None:
            raise ValueError("Model has not been fitted yet!")
        
        # Linear regression equation: y = b0 + b1*x1 + b2*x2 + ... + bn*xn
        predictions = self.intercept + X @ self.coefficients
        return predictions
    
    def score(self, X, y):
        """Calculate R² score on given data."""
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        return r2

print("LinearRegressionNumPy class defined successfully!")

## Step 4: Train the Model

In [ ]:
# Create and train the model
model = LinearRegressionNumPy()
model.fit(X_train, y_train)

print("\n" + "="*50)
print("Model Coefficients (Feature Weights):")
print("="*50)

# Get feature names
feature_names = [col for col in df.drop('exam_score', axis=1).columns]

# Display coefficients sorted by absolute value
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model.coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_df.to_string(index=False))

## Step 5: Model Evaluation

In [ ]:
# Make predictions on train and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate evaluation metrics
def calculate_metrics(y_true, y_pred, dataset_name):
    """Calculate and display evaluation metrics."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{dataset_name} Metrics:")
    print(f"  Mean Squared Error (MSE):     {mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"  Mean Absolute Error (MAE):    {mae:.4f}")
    print(f"  R² Score:                     {r2:.4f}")
    
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

print("="*50)
print("Model Performance Metrics")
print("="*50)

train_metrics = calculate_metrics(y_train, y_train_pred, "Training Set")
test_metrics = calculate_metrics(y_test, y_test_pred, "Testing Set")

## Step 6: Visualizations

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Actual vs Predicted (Training Set)
axes[0, 0].scatter(y_train, y_train_pred, alpha=0.6, color='blue', label='Predictions')
axes[0, 0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Exam Score')
axes[0, 0].set_ylabel('Predicted Exam Score')
axes[0, 0].set_title(f'Training Set: Actual vs Predicted (R² = {train_metrics["R2"]:.4f})')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Actual vs Predicted (Testing Set)
axes[0, 1].scatter(y_test, y_test_pred, alpha=0.6, color='green', label='Predictions')
axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual Exam Score')
axes[0, 1].set_ylabel('Predicted Exam Score')
axes[0, 1].set_title(f'Testing Set: Actual vs Predicted (R² = {test_metrics["R2"]:.4f})')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Residuals Plot (Training Set)
train_residuals = y_train - y_train_pred
axes[1, 0].scatter(y_train_pred, train_residuals, alpha=0.6, color='blue')
axes[1, 0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 0].set_xlabel('Predicted Exam Score')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('Training Set: Residual Plot')
axes[1, 0].grid(True, alpha=0.3)

# 4. Residuals Plot (Testing Set)
test_residuals = y_test - y_test_pred
axes[1, 1].scatter(y_test_pred, test_residuals, alpha=0.6, color='green')
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Predicted Exam Score')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Testing Set: Residual Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_evaluation.png', dpi=100, bbox_inches='tight')
plt.show()

print("Visualization saved as 'model_evaluation.png'")

In [ ]:
# Feature Importance Visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Get top features by absolute coefficient value
top_n = 10
coef_sorted = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model.coefficients
}).reindex(pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model.coefficients
}).sort_values('Coefficient', key=abs, ascending=False).index)

coef_top = coef_sorted.head(top_n)

colors = ['green' if x > 0 else 'red' for x in coef_top['Coefficient']]
ax.barh(coef_top['Feature'], coef_top['Coefficient'], color=colors, alpha=0.7)
ax.set_xlabel('Coefficient Value')
ax.set_title('Top 10 Feature Importance in Linear Regression Model')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=100, bbox_inches='tight')
plt.show()

print("Feature importance visualization saved as 'feature_importance.png'")

In [ ]:
# Distribution of residuals
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Training residuals histogram
axes[0].hist(train_residuals, bins=20, color='blue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Residuals')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Training Residuals Distribution\n(Mean: {train_residuals.mean():.4f}, Std: {train_residuals.std():.4f})')
axes[0].grid(True, alpha=0.3)

# Testing residuals histogram
axes[1].hist(test_residuals, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Testing Residuals Distribution\n(Mean: {test_residuals.mean():.4f}, Std: {test_residuals.std():.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('residuals_distribution.png', dpi=100, bbox_inches='tight')
plt.show()

print("Residuals distribution saved as 'residuals_distribution.png'")

## Step 7: Model Summary and Insights

In [ ]:
print("\n" + "="*60)
print("EXAM SCORE PREDICTION MODEL - FINAL SUMMARY")
print("="*60)

print("\n📊 DATASET INFORMATION:")
print(f"   • Total Samples: {len(df)}")
print(f"   • Training Samples: {len(X_train)}")
print(f"   • Testing Samples: {len(X_test)}")
print(f"   • Number of Features: {X_train.shape[1]}")
print(f"   • Target Variable: exam_score")

print("\n🎯 MODEL ARCHITECTURE:")
print(f"   • Algorithm: Linear Regression (Normal Equation)")
print(f"   • Implementation: NumPy")
print(f"   • Equation: y = {model.intercept:.4f} + Σ(β_i × x_i)")

print("\n📈 TRAINING PERFORMANCE:")
print(f"   • R² Score: {train_metrics['R2']:.4f}")
print(f"   • RMSE: {train_metrics['RMSE']:.4f}")
print(f"   • MAE: {train_metrics['MAE']:.4f}")

print("\n🧪 TESTING PERFORMANCE:")
print(f"   • R² Score: {test_metrics['R2']:.4f}")
print(f"   • RMSE: {test_metrics['RMSE']:.4f}")
print(f"   • MAE: {test_metrics['MAE']:.4f}")

print("\n🔝 TOP 5 MOST IMPORTANT FEATURES (by coefficient magnitude):")
top_features = coef_df.head(5)
for idx, row in top_features.iterrows():
    print(f"   {idx+1}. {row['Feature']:20s}: {row['Coefficient']:8.4f}")

print("\n💡 KEY INSIGHTS:")
print(f"   • Model explains {test_metrics['R2']*100:.2f}% variance in exam scores")
print(f"   • Average prediction error: ±{test_metrics['MAE']:.2f} points")
print(f"   • Model is {'GOOD' if test_metrics['R2'] > 0.7 else 'MODERATE' if test_metrics['R2'] > 0.5 else 'POOR'}")
print(f"   • No significant overfitting detected" if abs(train_metrics['R2'] - test_metrics['R2']) < 0.1 else "   • Some overfitting detected")

print("\n" + "="*60)

## Step 8: Example Predictions

In [ ]:
# Display sample predictions
print("\n📋 Sample Predictions from Test Set:")
print("="*80)

sample_indices = np.random.choice(len(y_test), 10, replace=False)
sample_df = pd.DataFrame({
    'Actual Score': y_test[sample_indices].round(2),
    'Predicted Score': y_test_pred[sample_indices].round(2),
    'Error': (y_test[sample_indices] - y_test_pred[sample_indices]).round(2),
    'Error %': ((abs(y_test[sample_indices] - y_test_pred[sample_indices]) / y_test[sample_indices]) * 100).round(2)
})

print(sample_df.to_string(index=False))
print("="*80)
print(f"\nNote: Error % shows the percentage difference between actual and predicted scores")